In [12]:
import numpy as np
from keras import layers, models
from keras.datasets import reuters
import matplotlib.pyplot as plt

# Obtener datos

In [2]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [3]:
word_index = reuters.get_word_index()
word_index = dict([(value, key) for (key, value) in word_index.items()])
for _ in train_data[0]:
    print(word_index.get(_-3))
    # pass

None
None
None
said
as
a
result
of
its
december
acquisition
of
space
co
it
expects
earnings
per
share
in
1987
of
1
15
to
1
30
dlrs
per
share
up
from
70
cts
in
1986
the
company
said
pretax
net
should
rise
to
nine
to
10
mln
dlrs
from
six
mln
dlrs
in
1986
and
rental
operation
revenues
to
19
to
22
mln
dlrs
from
12
5
mln
dlrs
it
said
cash
flow
per
share
this
year
should
be
2
50
to
three
dlrs
reuter
3


In [4]:
train_labels[0]

3

# Transformar datos a tensores

In [5]:
train_data.shape, test_data.shape
# No son tensores

((8982,), (2246,))

In [6]:
def vectorizar(sequences, dim=10000):
    result = np.zeros((len(sequences), dim))
    for i, sequences in enumerate(sequences):
        result[i, sequences]=1
    return result

x_train = vectorizar(train_data)
x_test = vectorizar(test_data)
# Son tensores
x_train.shape, x_test.shape

((8982, 10000), (2246, 10000))

In [7]:
from keras.utils import to_categorical
train_labels[5]

4

In [8]:
# Transformar a estructura de tensor
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)
y_train[5]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

# Crear red

In [9]:
model1 = models.Sequential()
model1.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model1.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model1.add(layers.Dense(46, activation='softmax')) # softmax -> usado para clasificacion multiple

model1.compile(optimizer="rmsprop",
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [10]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = y_train[:1000]
partial_y_train = y_train[1000:]

In [11]:
# Entrenamiento de modelo
history = model1.fit(partial_x_train, partial_y_train,
                    epochs=30,                          # Iteraciones
                    batch_size=512,                     # lotes de datos
                    validation_data=(x_val, y_val))

Epoch 1/30
16/16 [==============================] - 2s 44ms/step - loss: 2.6952 - accuracy: 0.4875 - val_loss: 1.7405 - val_accuracy: 0.6320
Epoch 2/30
16/16 [==============================] - 0s 21ms/step - loss: 1.4295 - accuracy: 0.7093 - val_loss: 1.3060 - val_accuracy: 0.7140
Epoch 3/30
16/16 [==============================] - 0s 25ms/step - loss: 1.0633 - accuracy: 0.7734 - val_loss: 1.1534 - val_accuracy: 0.7450
Epoch 4/30
16/16 [==============================] - 0s 27ms/step - loss: 0.8477 - accuracy: 0.8136 - val_loss: 1.0496 - val_accuracy: 0.7800
Epoch 5/30
16/16 [==============================] - 0s 26ms/step - loss: 0.6814 - accuracy: 0.8534 - val_loss: 0.9847 - val_accuracy: 0.7880
Epoch 6/30
16/16 [==============================] - 0s 26ms/step - loss: 0.5444 - accuracy: 0.8879 - val_loss: 0.9487 - val_accuracy: 0.8020
Epoch 7/30
16/16 [==============================] - 0s 30ms/step - loss: 0.4395 - accuracy: 0.9089 - val_loss: 0.9032 - val_accuracy: 0.8130
Epoch 8/30
16

In [13]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epoch = range(1, len(history_dict['loss'])+1)
plt.plot(epoch, history_dict['loss'], 'o', label='training')
plt.plot(epoch, val_loss_values, '--',label='val')
plt.legend()
plt.show()

: 

: 